pip install boto3

In [19]:
import boto3
import os
import re

In [2]:
aws_access_key_id = "xxxxxxxxxxxxxxxxx"
aws_secret_access_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [4]:
def upload_csv_to_s3(bucket_name, file_path):
    object_url = None
    response = None
    try:
        if os.path.split(file_path)[1][-3:] != "csv":
            raise Exception("Not a Valid File. Please upload the csv file only")
        with open(file_path,"rb") as fr:
            bytes = fr.read()
            s3_client = boto3.client('s3',
                                     aws_access_key_id=aws_access_key_id,
                                     aws_secret_access_key=aws_secret_access_key)
            file_name = os.path.split(file_path)[1]
            response = s3_client.put_object(Bucket=bucket_name, Key= file_name , Body=bytes)
            object_url = "s3://{}/{}".format(bucket_name, file_name)
            if response['ResponseMetadata']['HTTPStatusCode'] == 200:
                print("File uploaded successfully!",object_url)
            else:
                print("Failed to upload file.") 
    except Exception as e:
        print(e)
    return object_url,response

In [5]:
def upload_csv_to_s3(bucket_name, file_path):
    object_url = None
    try:
        if os.path.split(file_path)[1][-3:] != "csv":
            raise Exception("Not a Valid File. Please upload the csv file only")
        s3_client = boto3.client('s3',
                                 aws_access_key_id=aws_access_key_id,
                                 aws_secret_access_key=aws_secret_access_key)
        file_name = os.path.split(file_path)[1]
        s3_client.upload_file(file_path, bucket_name, file_name)
        object_url = "s3://{}/{}".format(bucket_name, file_name)
        print("File uploaded successfully!",object_url)
    except Exception as e:
        print(e)
    return object_url,None

In [6]:
bucket_name = "personalize-sampledatabucket-1"
file_path_item = "./Sample_Personalize_Data/item-meta-modified.csv"
file_path_interactions = "./Sample_Personalize_Data/interactions.csv"


s3_path_item,_ = upload_csv_to_s3(bucket_name, file_path_item)
s3_path_interactions,_ = upload_csv_to_s3(bucket_name, file_path_interactions)

File uploaded successfully! s3://personalize-sampledatabucket-1/v1_item-meta-modified.csv
File uploaded successfully! s3://personalize-sampledatabucket-1/v1_interactions.csv


In [7]:
print(s3_path_item,s3_path_interactions)

s3://personalize-sampledatabucket-1/v1_item-meta-modified.csv s3://personalize-sampledatabucket-1/v1_interactions.csv


In [8]:
os.path.split(s3_path_item)[1][:-4]

'v1_item-meta-modified'

In [9]:
def import_dataset_to_personalize(s3_path, datasetArn, region_name, mode = "INCREMENTAL"):
    personalize = boto3.client('personalize',
                               aws_access_key_id=aws_access_key_id,
                               aws_secret_access_key=aws_secret_access_key,
                               region_name=region_name)
    
    response = personalize.create_dataset_import_job(
        jobName = "import_job-"+os.path.split(s3_path)[1][:-4],
        datasetArn = datasetArn,
        dataSource = {'dataLocation':s3_path},
        roleArn = 'arn:aws:iam::973595392445:role/personalizerole-sampledatabucket-1',
        importMode = mode
    )
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        print("import job created successfully!")
    else:
        print("Failed to create import job.") 
    return response

In [10]:
datasetArn_items = 'arn:aws:personalize:us-west-2:973595392445:dataset/Giftzaza_Trail/ITEMS'
import_dataset_to_personalize(s3_path_item,datasetArn_items,region_name = "us-west-2")

import job created successfully!


{'datasetImportJobArn': 'arn:aws:personalize:us-west-2:973595392445:dataset-import-job/import_job-v1_item-meta-modified',
 'ResponseMetadata': {'RequestId': 'a7befaa4-8c61-4058-a063-54bcbf00f0a7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 17 Jan 2024 12:13:43 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '120',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a7befaa4-8c61-4058-a063-54bcbf00f0a7'},
  'RetryAttempts': 0}}

In [11]:
datasetArn_interactions = 'arn:aws:personalize:us-west-2:973595392445:dataset/Giftzaza_Trail/INTERACTIONS'
import_dataset_to_personalize(s3_path_interactions,datasetArn_interactions,region_name = "us-west-2")

import job created successfully!


{'datasetImportJobArn': 'arn:aws:personalize:us-west-2:973595392445:dataset-import-job/import_job-v1_interactions',
 'ResponseMetadata': {'RequestId': '072ef1c6-3752-49e5-a5db-e4521355f331',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 17 Jan 2024 12:13:45 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '114',
   'connection': 'keep-alive',
   'x-amzn-requestid': '072ef1c6-3752-49e5-a5db-e4521355f331'},
  'RetryAttempts': 0}}

In [12]:
personalize = boto3.client('personalize',
                               aws_access_key_id=aws_access_key_id,
                               aws_secret_access_key=aws_secret_access_key,
                               region_name="us-west-2")
response = personalize.describe_dataset_import_job(
    datasetImportJobArn='arn:aws:personalize:us-west-2:973595392445:dataset-import-job/import_job-v1_interactions'
)
print(response['datasetImportJob']['status'],response)

CREATE IN_PROGRESS {'datasetImportJob': {'jobName': 'import_job-v1_interactions', 'datasetImportJobArn': 'arn:aws:personalize:us-west-2:973595392445:dataset-import-job/import_job-v1_interactions', 'datasetArn': 'arn:aws:personalize:us-west-2:973595392445:dataset/Giftzaza_Trail/INTERACTIONS', 'dataSource': {'dataLocation': 's3://personalize-sampledatabucket-1/v1_interactions.csv'}, 'roleArn': 'arn:aws:iam::973595392445:role/personalizerole-sampledatabucket-1', 'status': 'CREATE IN_PROGRESS', 'creationDateTime': datetime.datetime(2024, 1, 17, 17, 43, 45, 377000, tzinfo=tzlocal()), 'lastUpdatedDateTime': datetime.datetime(2024, 1, 17, 17, 44, 2, 977000, tzinfo=tzlocal()), 'importMode': 'INCREMENTAL', 'publishAttributionMetricsToS3': False}, 'ResponseMetadata': {'RequestId': '3e2be5a0-af55-453b-8849-5c1565aa5c7d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 17 Jan 2024 12:15:03 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '604', 'connection': 'keep-alive',

In [13]:
def create_solution_version(solution_name,solutionArn,region_name,mode = 'UPDATE'):
    personalize = boto3.client('personalize',
                               aws_access_key_id=aws_access_key_id,
                               aws_secret_access_key=aws_secret_access_key,
                               region_name=region_name)
    response = personalize.create_solution_version(
    name=solution_name,
    solutionArn=solutionArn,
    trainingMode=mode
    )
    return response

In [14]:
solution_name = "solutionV1"
solutionArn = "arn:aws:personalize:us-west-2:973595392445:solution/Personalize-UserSpecificRecommendation-1"

create_solution_version(solution_name,solutionArn,region_name = "us-west-2")

{'solutionVersionArn': 'arn:aws:personalize:us-west-2:973595392445:solution/Personalize-UserSpecificRecommendation-1/solutionV1',
 'ResponseMetadata': {'RequestId': '44c3774e-e174-4308-8198-f85edd26c206',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 17 Jan 2024 12:21:24 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '128',
   'connection': 'keep-alive',
   'x-amzn-requestid': '44c3774e-e174-4308-8198-f85edd26c206'},
  'RetryAttempts': 0}}

In [17]:
personalize = boto3.client('personalize',
                               aws_access_key_id=aws_access_key_id,
                               aws_secret_access_key=aws_secret_access_key,
                               region_name="us-west-2")
response = personalize.describe_solution_version(
    solutionVersionArn='arn:aws:personalize:us-west-2:973595392445:solution/Personalize-UserSpecificRecommendation-1/solutionV1'
)
print(response)

{'solutionVersion': {'name': 'Personalize-UserSpecificRecommendation-1/solutionV1', 'solutionVersionArn': 'arn:aws:personalize:us-west-2:973595392445:solution/Personalize-UserSpecificRecommendation-1/solutionV1', 'solutionArn': 'arn:aws:personalize:us-west-2:973595392445:solution/Personalize-UserSpecificRecommendation-1', 'performHPO': False, 'performAutoML': False, 'recipeArn': 'arn:aws:personalize:::recipe/aws-user-personalization', 'datasetGroupArn': 'arn:aws:personalize:us-west-2:973595392445:dataset-group/Giftzaza_Trail', 'solutionConfig': {'algorithmHyperParameters': {'bptt': '32', 'hidden_dimension': '149', 'recency_mask': 'true'}, 'featureTransformationParameters': {'max_user_history_length_percentile': '0.99', 'min_user_history_length_percentile': '0.00'}}, 'trainingMode': 'UPDATE', 'status': 'CREATE IN_PROGRESS', 'creationDateTime': datetime.datetime(2024, 1, 17, 17, 51, 24, 620000, tzinfo=tzlocal()), 'lastUpdatedDateTime': datetime.datetime(2024, 1, 17, 17, 51, 44, 659000, t

In [18]:
def Update_campaign(campaignArn,solutionVersionArn,region_name,minProvisionedTPS=1):
    personalize = boto3.client('personalize',
                               aws_access_key_id=aws_access_key_id,
                               aws_secret_access_key=aws_secret_access_key,
                               region_name=region_name)
    response = personalize.update_campaign(
        campaignArn=campaignArn,
        solutionVersionArn=solutionVersionArn,
        minProvisionedTPS=1
    )
    return response

In [19]:
campaignArn = "arn:aws:personalize:us-west-2:973595392445:campaign/Personalize-UserSpecificRecommendation-Camp1"
new_solutionVersionArn = 'arn:aws:personalize:us-west-2:973595392445:solution/Personalize-UserSpecificRecommendation-1/solutionV1'

In [20]:
Update_campaign(campaignArn,new_solutionVersionArn,region_name="us-west-2")

{'campaignArn': 'arn:aws:personalize:us-west-2:973595392445:campaign/Personalize-UserSpecificRecommendation-Camp1',
 'ResponseMetadata': {'RequestId': 'bafac56a-c4be-4df1-bf48-c06244ecee2d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 17 Jan 2024 12:33:42 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '114',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'bafac56a-c4be-4df1-bf48-c06244ecee2d'},
  'RetryAttempts': 0}}

In [3]:
def get_solution_metrics(solutionVersionArn):
    personalize = boto3.client('personalize',
                               aws_access_key_id=aws_access_key_id,
                               aws_secret_access_key=aws_secret_access_key,
                               region_name="us-west-2")
    response = personalize.get_solution_metrics(
        solutionVersionArn= solutionVersionArn
    )
    return response

In [9]:
new_solutionVersionArn = 'arn:aws:personalize:us-west-2:973595392445:solution/Personalize-UserSpecificRecommendation-1/solutionV1'

In [10]:
sol_m1 = get_solution_metrics(new_solutionVersionArn)

In [11]:
sol_m1

{'solutionVersionArn': 'arn:aws:personalize:us-west-2:973595392445:solution/Personalize-UserSpecificRecommendation-1/solutionV1',
 'metrics': {'coverage': 0.0772,
  'mean_reciprocal_rank_at_25': 0.2094,
  'normalized_discounted_cumulative_gain_at_10': 0.216,
  'normalized_discounted_cumulative_gain_at_25': 0.2828,
  'normalized_discounted_cumulative_gain_at_5': 0.1735,
  'precision_at_10': 0.0375,
  'precision_at_25': 0.0286,
  'precision_at_5': 0.0464},
 'ResponseMetadata': {'RequestId': '49692605-ecbc-4da7-a783-239e9ded4669',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 22 Jan 2024 05:55:56 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '425',
   'connection': 'keep-alive',
   'x-amzn-requestid': '49692605-ecbc-4da7-a783-239e9ded4669'},
  'RetryAttempts': 0}}

In [12]:
sol_m2 = get_solution_metrics("arn:aws:personalize:us-west-2:973595392445:solution/Personalize-UserSpecificRecommendation-1/Personalize-UserSpecificRecommendation-1")

In [13]:
sol_m2

{'solutionVersionArn': 'arn:aws:personalize:us-west-2:973595392445:solution/Personalize-UserSpecificRecommendation-1/Personalize-UserSpecificRecommendation-1',
 'metrics': {'coverage': 0.0776,
  'mean_reciprocal_rank_at_25': 0.2097,
  'normalized_discounted_cumulative_gain_at_10': 0.2147,
  'normalized_discounted_cumulative_gain_at_25': 0.2863,
  'normalized_discounted_cumulative_gain_at_5': 0.1773,
  'precision_at_10': 0.0357,
  'precision_at_25': 0.0293,
  'precision_at_5': 0.0464},
 'ResponseMetadata': {'RequestId': '79b613f6-3392-44c0-ab7c-10e3a42027f9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 22 Jan 2024 05:56:02 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '456',
   'connection': 'keep-alive',
   'x-amzn-requestid': '79b613f6-3392-44c0-ab7c-10e3a42027f9'},
  'RetryAttempts': 0}}

In [53]:
def CompareMetrics(old_solm,new_solm):
    flag = True
    for key,value in new_solm.items():
        if re.search("^precision.*",key):
            delta = round(new_solm[key]-old_solm[key],3)
            if delta < 0:
                print(key,delta)
                flag = False
    return flag

In [54]:
CompareMetrics(sol_m2['metrics'],sol_m1['metrics'])

precision_at_25 -0.001


False